# Creation of the Raw Datasets using Developer's API

The purpose of this notebook is to create the two main datasets needed to estimate the effect of investor sentiment on USD/MXN Exchange rate volatility:

- Time series of USD/MXN FX rate.
- A dense dataset with news that affect investors perception on Mexican financials.

In [2]:
import pandas as pd
import requests

from datetime import datetime, timedelta
from pathlib import Path

## 1. New York Times API: news dataset for Sentiment Analysis

Using the NYT API I will query all of the news articles related to Mexico since Jan 1, 1996.
Before running the cell below you need: 
- to create a Developer API key [here](https://developer.nytimes.com/).
- save it as `nyt_api_key.txt` in your credentials folder

In [3]:
import time
from copy import copy
from math import floor

API_ENDPOINT = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
API_KEY = Path('credentials','nyt_api_key.txt').read_text()

API_PARAMS = {
    'fq': 'glocations.contains:("MEXICO" NOT "NEW")'
    + 'AND type_of_material.contains:("News")',
    'sort': 'newest',
    'api-key': API_KEY,
    'page': 0,
    'begin_date': None,
    'end_date': None
}

HITS_PER_PAGE = 10
YEARS = [*range(1996,2023,3)]
MAX_DATE = datetime.now()

dates = [datetime(year, 1, 1) for year in YEARS]
articles = []

There are two rate limits per API: 
- 4,000 requests per day 
- 10 requests per minute. 

The loop sleeps 6 seconds between calls to avoid hitting the per minute rate limit.<br/><br/>
Also, the API returns only 10 results per page and will not paginate past page 100, it's necessary to do a double-loop in which the request is made for a limited time period in which the total number of results is less than 1000. It takes over an hour to run.

In [6]:
a = time.time()
for i, dt in enumerate(dates):
    
    begin_date = dt
    end_date = (dates[i+1] - timedelta(days=1) if i < len(dates) - 1 else MAX_DATE)
    
    parms = copy(API_PARAMS)
    parms['begin_date'] = begin_date.strftime('%Y%m%d')
    parms['end_date'] = end_date.strftime('%Y%m%d')

    print("\n{0} to {1}".format(begin_date, end_date))
    parms['page'] = 0
    total_hits = -1
    
    while total_hits == -1 or parms['page'] * HITS_PER_PAGE < total_hits:
        print("\tPage {0} of {1}; hits: {2}".format(parms['page'],
                                                    floor(total_hits / HITS_PER_PAGE),
                                                    total_hits,))
        
        resp = requests.get(API_ENDPOINT, params=parms, 
                #verify = 'credentials/mycerts.pem'
                )
        try:
            r = resp.json()['response']
            articles.extend(r['docs'])
            total_hits = r['meta']['hits']
            parms['page'] += 1
            time.sleep(5.6)
        except:
            "No response"
            
b = time.time()

print(b-a)
        
    


1996-01-01 00:00:00 to 1998-12-31 00:00:00
	Page 0 of -1; hits: -1
	Page 1 of 92; hits: 925
	Page 2 of 92; hits: 925
	Page 3 of 92; hits: 925
	Page 4 of 92; hits: 925
	Page 5 of 92; hits: 925
	Page 6 of 92; hits: 925
	Page 7 of 92; hits: 925
	Page 8 of 92; hits: 925
	Page 9 of 92; hits: 925
	Page 10 of 92; hits: 925
	Page 11 of 92; hits: 925
	Page 12 of 92; hits: 925
	Page 13 of 92; hits: 925
	Page 14 of 92; hits: 925
	Page 15 of 92; hits: 925
	Page 16 of 92; hits: 925
	Page 17 of 92; hits: 925
	Page 18 of 92; hits: 925
	Page 19 of 92; hits: 925
	Page 20 of 92; hits: 925
	Page 21 of 92; hits: 925
	Page 22 of 92; hits: 925
	Page 23 of 92; hits: 925
	Page 24 of 92; hits: 925
	Page 25 of 92; hits: 925
	Page 26 of 92; hits: 925
	Page 27 of 92; hits: 925
	Page 28 of 92; hits: 925
	Page 29 of 92; hits: 925
	Page 30 of 92; hits: 925
	Page 31 of 92; hits: 925
	Page 32 of 92; hits: 925
	Page 33 of 92; hits: 925
	Page 34 of 92; hits: 925
	Page 35 of 92; hits: 925
	Page 36 of 92; hits: 925
	Page

Now we have a list named `articles` with >6,000 python dictionaries. We need to parse it before converting to a DataFrame

In [7]:
key = 'print_section'
n = len(articles)
str_int_keys = {*articles[i].keys()}-{'keywords','headline','byline','multimedia'}

def extract_from_list_of_dics(list,i,key):
    try:
        return list[i][key]
    except: 
        return ''

dic = {key : [extract_from_list_of_dics(articles,i,key) for i in range(n)] for key in str_int_keys}

dic2 = {
    'headline': [extract_from_list_of_dics(articles,i,'headline')['main'] for i in range(n)],
    'author' : [extract_from_list_of_dics(articles,i,'byline')['original'] for i in range(n)]
}

dic.update(dic2)

Finally we convert to a `df` and save to a csv file

In [9]:
nyt = pd.DataFrame(dic)
nyt.to_csv('csv_files/nyt.csv', index = False)

nyt.head()

,snippet,abstract,pub_date,_id,section_name,uri,type_of_material,source,print_section,lead_paragraph,word_count,news_desk,web_url,document_type,print_page,headline,author
0,Mexican lawmakers reach compromise on country'...,Mexican lawmakers reach compromise on country'...,1998-12-31T05:00:00+0000,nyt://article/6238be98-a87a-5fbd-8a5d-20b15341...,World,nyt://article/6238be98-a87a-5fbd-8a5d-20b15341...,News,The New York Times,A,After weeks of tense impasse and with less tha...,518,Foreign Desk,https://www.nytimes.com/1998/12/31/world/just-...,article,9,"Just Hours Before Deadline, Mexicans Draft an ...",By Ginger Thompson
1,Three new studies report new technique to pinp...,Three new studies report new technique to pinp...,1998-12-29T05:00:00+0000,nyt://article/bd31e0fe-4053-55f8-9e24-86b10281...,Science,nyt://article/bd31e0fe-4053-55f8-9e24-86b10281...,News,The New York Times,F,"Every winter, after flying south from the East...",1170,Science Desk,https://www.nytimes.com/1998/12/29/science/on-...,article,5,"On the Trail of the Monarch, With the Aid of C...",By Carol Kaesuk Yoon
2,Interview with Lieut Col Hildegardo Bacillo Go...,Interview with Lieut Col Hildegardo Bacillo Go...,1998-12-29T05:00:00+0000,nyt://article/fc8cfb88-aa41-5ec5-8328-f2e6d870...,World,nyt://article/fc8cfb88-aa41-5ec5-8328-f2e6d870...,News,The New York Times,A,Sheltered from view by bushes on the side of t...,1352,Foreign Desk,https://www.nytimes.com/1998/12/29/world/mexic...,article,3,Mexican Army Protester Goes Loudly Into Hiding,By Ginger Thompson
3,"Some 2,000 people gather at graves of 45 peopl...","Some 2,000 people gather at graves of 45 peopl...",1998-12-23T05:00:00+0000,nyt://article/1eaf78c0-88df-5390-bd95-664fcf07...,World,nyt://article/1eaf78c0-88df-5390-bd95-664fcf07...,News,The New York Times,A,"For six days, only the dogs and chickens were ...",951,Foreign Desk,https://www.nytimes.com/1998/12/23/world/actea...,article,4,Acteal Journal; Where Killings Defiled a Churc...,By Ginger Thompson
4,Ambitious United States effort to help train a...,Ambitious United States effort to help train a...,1998-12-23T05:00:00+0000,nyt://article/aae2ecc4-f440-5989-bdab-6e56f562...,World,nyt://article/aae2ecc4-f440-5989-bdab-6e56f562...,News,The New York Times,A,An ambitious United States effort to help trai...,1261,Foreign Desk,https://www.nytimes.com/1998/12/23/world/us-he...,article,1,U.S. Help for Mexican Military Has Not Curtail...,By Tim Golden


## 2. Banxico: USD/Peso FX

Using the Banxico SIE API I will query the historical daily USD/MXN Peso Exchange Rate Value (Tipo de cambio para solventar obligaciones denominadas en dólares Fecha de liquidación `SF60653`).
Before running the cell below you need: 
- to create a Developer API token [here](https://www.banxico.org.mx/SieAPIRest/service/v1/).
- save it as `banxico_token.txt` in your credentials folder

In [10]:
token = Path('credentials','banxico_token.txt').read_text()
response = requests.get(
    'https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF60653/datos',
    params={
        'token': token
    }
)
        
bmx = response.json()

We need to parse the JSON before converting into a DataFrame

In [11]:
bmx_dic = {'date': [bmx['bmx']['series'][0]['datos'][i]['fecha'] for i in range(len(bmx['bmx']['series'][0]['datos']))],
           'fx' : [bmx['bmx']['series'][0]['datos'][i]['dato'] for i in range(len(bmx['bmx']['series'][0]['datos']))]}

Create the csv file from the DataFrame

In [12]:
bmx_df = pd.DataFrame.from_dict(bmx_dic)
bmx_df.to_csv('csv_files/bmx.csv', index = False)
bmx_df.head()

,date,fx
0,14/11/1991,3.0735
1,15/11/1991,3.0712
2,16/11/1991,3.0718
3,17/11/1991,3.0718
4,18/11/1991,3.0718


## Appendix: Twitter dataset

In [1]:
from pathlib import Path
from twython import Twython

ACCESS_TOKEN, APP_KEY = [Path('credentials', f).read_text() for f in ['tw_access_token.txt','tw_app_key.txt']]

In [2]:
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [3]:
tweet = twitter.search(q='mexico', lang = 'en', result_type = 'popular',count = 10)
tweet_list = [tweet['statuses'][i]['text'] for i in range(len(tweet['statuses']))]